In [ ]:
import keras.backend
import keras.callbacks
import keras.layers
import keras.models
import keras.optimizers

import matplotlib
matplotlib.use('SVG')

import helper.callbacks
import helper.model_builder
import helper.visualize
import helper.data_provider
import helper.metrics

import skimage.io
import sklearn.metrics

import scipy.stats
import pandas as pd

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import sys

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

Using TensorFlow backend.


In [ ]:
# constants
const_lr = 1e-4

tag = '01'

# Output dirs
out_dir = '/data1/image-segmentation/BBBC022/unet/experiments/' + tag + '/out'
tb_log_dir = "/data1/image-segmentation/BBBC022/unet/tensorboard/" + tag + "/"

os.makedirs(out_dir, exist_ok=True)
os.makedirs(tb_log_dir, exist_ok=True)

# Files
chkpt_file = "/data1/image-segmentation/BBBC022/unet/experiments/" + tag + "/checkpoints/{epoch:04d}.hdf5"
csv_log_file = "/data1/image-segmentation/BBBC022/unet/experiments/" + tag + "/log.csv"

# Input dirs
train_dir_x = '/data1/image-segmentation/BBBC022/unet/split/training/x/'
train_dir_y = '/data1/image-segmentation/BBBC022/unet/split/training/y/'
val_dir = "/data1/image-segmentation/BBBC022/unet/split/validation/"

# Learning Settings
rescale_labels = False

epochs = 2

batch_size = 4
steps_per_epoch = 2 * 4 / batch_size

# make sure these number for to the validation set
val_batch_size = 10
val_steps = int(50 * 4 / val_batch_size)

# generator only params
dim1 = 256
dim2 = 256

bit_depth = 8

In [ ]:
# build session running on GPU 1
configuration = tf.ConfigProto()
#configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = ""
session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)

train_gen = helper.data_provider.random_sample_generator(
    train_dir_x,
    train_dir_y,
    batch_size,
    bit_depth,
    dim1,
    dim2,
    rescale_labels
)

val_gen = helper.data_provider.single_data_from_images(
    val_dir + 'x/',
     val_dir + 'y/',
     val_batch_size,
     bit_depth,
     dim1,
     dim2,
     rescale_labels
)

In [ ]:
# build model
model = helper.model_builder.get_model_3_class(dim1, dim2)
model.summary()

loss = "categorical_crossentropy"
metrics = [keras.metrics.categorical_accuracy, helper.metrics.recall, helper.metrics.precision]

optimizer = keras.optimizers.RMSprop(lr = const_lr)

model.compile(loss=loss, metrics=metrics, optimizer=optimizer)

# CALLBACKS
# save model after each epoch
callback_model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=chkpt_file,
    save_weights_only=True,
    save_best_only=False
)
callback_csv = keras.callbacks.CSVLogger(filename=csv_log_file)
# callback_splits_and_merges = helper.callbacks.SplitsAndMergesLoggerBoundary(
#     'images',
#     val_gen,
#     gen_calls = val_steps,
#     log_dir=tb_log_dir
# )

callbacks=[callback_model_checkpoint, callback_csv] #, callback_splits_and_merges]


In [ ]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_gen,
    validation_steps=val_steps,
    callbacks=callbacks,
    verbose = 1
)

# visualize learning stats
#helper.visualize.visualize_learning_stats_boundary_hard(statistics, out_dir, metrics)

print('Done! :)')